  # импорт библиотек

In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

# raw data / сырые данные

In [17]:
raw_data = pd.read_csv('movie_bd_v5.csv')
raw_data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1856,tt0163978,40000000,144056873,The Beach,Leonardo DiCaprio|Guillaume Canet|Tilda Swinto...,Danny Boyle,Somewhere on this planet it must exist.,Twenty-something Richard travels to Thailand a...,119,Drama|Adventure|Romance|Thriller,20th Century Fox|Figment Films,2/11/2000,6.2,2000
1239,tt3063516,15000000,151831537,Jackass Presents: Bad Grandpa,Johnny Knoxville|Jackson Nicoll|Georgina Cates...,Jeff Tremaine,Real people. Real reactions. Real messed up.,86-year-old Irving Zisman is on a journey acro...,92,Comedy,Paramount Pictures|MTV Films|Dickhouse Product...,10/22/2013,6.1,2013
1800,tt0209144,9000000,39723096,Memento,Guy Pearce|Carrie-Anne Moss|Joe Pantoliano|Mar...,Christopher Nolan,Some memories are best forgotten.,Suffering short-term memory loss after a head ...,113,Mystery|Thriller,Summit Entertainment|Newmarket Capital Group|T...,10/11/2000,7.9,2000
230,tt2281587,50000000,80383113,Muppets Most Wanted,Ricky Gervais|Ty Burrell|Tina Fey|Steve Whitmi...,James Bobin,Taking the world by farce,"While on a grand world tour, The Muppets find ...",112,Comedy|Adventure|Crime|Family,Buena Vista|Disney,3/20/2014,6.2,2014
1666,tt0359517,12000000,31179516,Johnson Family Vacation,Cedric the Entertainer|Vanessa Williams|Steve ...,Christopher Erskin,Take the ride.,AAA can't help the roadside emergency that is ...,97,Comedy|Family,Fox Searchlight Pictures|Bird and a Bear Enter...,4/7/2004,5.3,2004


In [19]:
raw_data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Колонки raw_data

In [20]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Нормализуем raw_data

### приводим столбцы к одному виду


genres : string - split('|')

cast : string - split('|')

director : string - split('|')

production_companies : string - split('|')

release_date : string - дата MM/DD/YYYY


### Добавляем столбцы

year : int - год релиза

month : int (1..12) - месяц релиза

date : int (1..31) - дата релиза

profit - выручка


In [24]:
# разделяет строку по вертикальной черте, 
# возвращает массив
def split_str_as_list( g ):
    if type(g) is str:
        return g.split('|')
    return []

In [25]:
def normalize_nf1( src_df, column, norm_f, newColumn=None, includeColumns=None ):
    """Нормализация DataFrame к 1 форме - случай split('|')
       Если в ячейки есть 2 или более значения, 
         1. то вся строка будет продублирова столько раз, сколько значейний в ячейке
         2. значение ячеек будет заменено на разультат разбияния

       Args:
         src_df - DataFrame
         column : str - Колонка содержащяя множествно значений
         norm_f( value ):List[values] - функция разбиения значения на список значений
         newColumn : str | None - переименование колонки
         includeColumns : None | List[str] - список колонок которые следует оставить

       Returs:
         Новый DataFrame

    """
    if not (type(src_df) is pd.DataFrame):
        raise "!! src_df not pd.DataFrame"
    if not (type(column) is str):
        raise "!! column not str"
    if not callable(norm_f):
        raise "!! norm_f not callable"    

    # Целевая колонка не указана - используем оригинальную
    if newColumn==None:
        newColumn = column
    elif not( type(newColumn) is str):
        raise "!! newColumn must None|str"

    # Строим новый массив данных
    column_store = {}
    for col_name in src_df.columns:
        column_store[col_name] = []
        column_store[col_name if col_name!=column else newColumn] = []

    # по строчное хранение
    row_store = []        
    for index, srow in src_df.iterrows():
        raw_unsplitted_data = srow[column]
        splitted_data = norm_f(raw_unsplitted_data)
        for splitted_value in splitted_data:
            row = {}
            for col_name in src_df.columns:
                row[col_name] = srow[col_name]
                row[col_name if col_name!=column else newColumn] = srow[col_name] if col_name != column else splitted_value
            row_store.append( row )
    
    for row in row_store:
        for k in column_store.keys():
            column_store[k].append(row[k])

    df = pd.DataFrame( column_store, index=range(0,len(row_store)) )

    # Грохаем лишние колонки
    if type(includeColumns) is list:
        cols = []
        keep_cols = []
        for col_name in df.columns:
            cols.append(col_name)
            if col_name == newColumn:
                keep_cols.append(col_name)
            elif col_name in includeColumns:
                keep_cols.append(col_name)
        drop_cols = [ c for c in cols if c not in keep_cols ]
        df = df.drop( drop_cols, axis=1 )

    return df

In [26]:
normalize_nf1( raw_data.head(3), 'genres', split_str_as_list, 'genre', ['imdb_id','genres','original_title'] )

,imdb_id,original_title,genres,genre
0,tt0369610,Jurassic World,Action|Adventure|Science Fiction|Thriller,Action
1,tt0369610,Jurassic World,Action|Adventure|Science Fiction|Thriller,Adventure
2,tt0369610,Jurassic World,Action|Adventure|Science Fiction|Thriller,Science Fiction
3,tt0369610,Jurassic World,Action|Adventure|Science Fiction|Thriller,Thriller
4,tt1392190,Mad Max: Fury Road,Action|Adventure|Science Fiction|Thriller,Action
5,tt1392190,Mad Max: Fury Road,Action|Adventure|Science Fiction|Thriller,Adventure
6,tt1392190,Mad Max: Fury Road,Action|Adventure|Science Fiction|Thriller,Science Fiction
7,tt1392190,Mad Max: Fury Road,Action|Adventure|Science Fiction|Thriller,Thriller
8,tt2908446,Insurgent,Adventure|Science Fiction|Thriller,Adventure
9,tt2908446,Insurgent,Adventure|Science Fiction|Thriller,Science Fiction


In [30]:
#   genre,acts .. - жанр,актер,режиссер,компания
#   imdb_id - ид фильма ..
genres = normalize_nf1( raw_data, 'genres', split_str_as_list, 'genre', ['imdb_id'] )
acts = normalize_nf1( raw_data, 'cast', split_str_as_list, 'actor', ['imdb_id'] )
dirs = normalize_nf1( raw_data, 'director', split_str_as_list, 'dir', ['imdb_id'] )
prods = normalize_nf1( raw_data, 'production_companies', split_str_as_list, 'prod', ['imdb_id'] )

In [33]:
# Копируем исходные данные
movies = raw_data.copy()
movies = movies.drop( ['genres','cast','director','production_companies'], axis=1 )

# Вычисление даты из строки формата MM/DD/YYYY
def date_str_to_values( s ):
    '''
    Вычисление даты из строки формата MM/DD/YYYY
    Args:
        s : str - строка с датой в формате MM/DD/YYYY
    Returns:
        { 'y': int # год
        , 'm': int # месяц
        , 'd': int # дата
        }
    '''
    nums = list(map( lambda x:int(x), s.split('/') ))
    return { 'y':nums[2], 'm':nums[0], 'd':nums[1] }

# Вычисляем колонки year, month, date
movies['year'] = movies['release_date'].apply( lambda x: date_str_to_values(x)['y'] )
movies['month'] = movies['release_date'].apply( lambda x: date_str_to_values(x)['m'] )
movies['date'] = movies['release_date'].apply( lambda x: date_str_to_values(x)['d'] )

movies = movies.drop( ['release_date'], axis=1 )

if( len(movies[ movies.release_year != movies.year ])>0 ):
    raise '!! movies.release_year != movies.year'

movies = movies.drop(['release_year'], axis=1)

movies['profit'] = movies['revenue'] - movies['budget']

# original_title_len - Длина заголовка в символах
movies['original_title_len'] = movies['original_title'].apply( lambda x: len(x) )

def simple_words_of( text ):
    return text.split(' ')

movies['overview_words_count'] = movies['overview'].apply( lambda x: len(simple_words_of(x)) )
movies


,imdb_id,budget,revenue,original_title,tagline,overview,runtime,vote_average,year,month,date,profit,original_title_len,overview_words_count
0,tt0369610,150000000,1513528810,Jurassic World,The park is open.,Twenty-two years after the events of Jurassic ...,124,6.5,2015,6,9,1363528810,14,26
1,tt1392190,150000000,378436354,Mad Max: Fury Road,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,7.1,2015,5,13,228436354,18,110
2,tt2908446,110000000,295238201,Insurgent,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,6.3,2015,3,18,185238201,9,22
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,7.5,2015,12,15,1868178225,28,26
4,tt2820852,190000000,1506249360,Furious 7,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,7.3,2015,4,1,1316249360,9,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,tt0120903,75000000,157299717,X-Men,Evolution Begins,"Two mutants, Rogue and Wolverine, come to a pr...",104,6.6,2000,7,13,82299717,5,27
1885,tt0192255,22000000,13555988,The Little Vampire,"They're not just best friends, they're blood b...","Based on the popular books, the story tells of...",95,6.4,2000,10,27,-8444012,18,75
1886,tt0131704,76000000,35134820,The Adventures of Rocky & Bullwinkle,This summer it's not the same old bull.,Rocky and Bullwinkle have been living off the ...,88,4.0,2000,6,30,-40865180,36,58
1887,tt0162983,40000000,36037909,Hanging Up,Every family has a few hang-ups.,A trio of sisters bond over their ambivalence ...,94,5.2,2000,2,16,-3962091,10,24


In [34]:

movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   tagline               1889 non-null   object 
 5   overview              1889 non-null   object 
 6   runtime               1889 non-null   int64  
 7   vote_average          1889 non-null   float64
 8   year                  1889 non-null   int64  
 9   month                 1889 non-null   int64  
 10  date                  1889 non-null   int64  
 11  profit                1889 non-null   int64  
 12  original_title_len    1889 non-null   int64  
 13  overview_words_count  1889 non-null   int64  
dtypes: float64(1), int64(9), object(4)
memory usage: 206.7+ KB


# Ответы на вопросы

In [36]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [37]:
r = movies[ movies.budget == movies.budget.max() ]
answers['1'] = r.original_title.iloc[0] + ' (' + r.imdb_id.iloc[0] + ')'
answers['1']

'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [38]:
r = movies[ movies.runtime == movies.runtime.max() ]
answers['2'] = r.original_title.iloc[0] + ' (' + r.imdb_id.iloc[0] + ')'
answers['2']

'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [39]:
r = movies[ movies.runtime == movies.runtime.min() ]
answers['3'] = r.original_title.iloc[0] + ' (' + r.imdb_id.iloc[0] + ')'
answers['3']

'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [40]:
answers['4'] = round(movies.runtime.mean())
answers['4']

110

# 5. Каково медианное значение длительности фильмов? 

In [41]:

answers['5'] = round(movies.runtime.median())
answers['5']

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [42]:
r = movies[ movies.profit == movies.profit.max() ]
answers['6'] = r.original_title.iloc[0] + ' (' + r.imdb_id.iloc[0] + ')'
answers['6']

'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [43]:

r = movies[ movies.profit == movies.profit.min() ]
answers['7'] = r.original_title.iloc[0] + ' (' + r.imdb_id.iloc[0] + ')'
answers['7']

'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [44]:
answers['8'] = len( movies[ movies.profit > 0 ] )
answers['8']

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [45]:
d = movies[ movies.year == 2008 ]
r = d[ d.revenue == d.revenue.max() ]
answers['9'] = r.original_title.iloc[0] + ' (' + r.imdb_id.iloc[0] + ')'
answers['9']

'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [46]:
d = movies[ (movies.year >= 2012) & (movies.year <= 2014) ]
r = d[ d.profit == d.profit.min() ]
answers['10'] = r.original_title.iloc[0] + ' (' + r.imdb_id.iloc[0] + ')'
answers['10']

'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [0]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

ВАРИАНТ 1

In [48]:
genres.describe()
answers['11'] = genres.describe().loc['top']['genre']
answers['11']

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [49]:
answers['12'] = genres.genre.value_counts().index[0]
answers['12']

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [50]:
j = dirs.merge( movies, on='imdb_id', how='outer' )
answers['13'] = j.groupby(['dir'])['revenue'].sum().sort_values(ascending=False).index[0]
answers['13']

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [51]:

j = genres.merge( movies, on='imdb_id', how='outer' )
j = j.merge( dirs, on='imdb_id', how='outer' )
j = j[ j.genre == 'Action' ]
answers['14'] = j.groupby( ['dir'] )['genre'].count().sort_values(ascending=False).index[0]
answers['14']

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [52]:
j = acts.merge( movies, on='imdb_id', how='outer' )
j = j[ j.year == 2012 ]
answers['15'] = j.groupby(['actor'])['revenue'].sum().sort_values(ascending=False).index[0]
answers['15']

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [53]:
j = acts.merge( movies, on='imdb_id', how='outer' )
# в фильмах, где бюджет выше среднего по данной выборке. 
j = j[ j.budget > j.budget.mean() ]
answers['16'] = j.groupby('actor').count().sort_values(['imdb_id'],ascending=False).index[0]
answers['16']

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [54]:
j = acts.merge( genres, on='imdb_id', how='outer' )
j = j[ j.actor=='Nicolas Cage' ].drop(['imdb_id'], axis=1)
j = j.groupby(['genre']).count().sort_values(['actor'],ascending=False)
answers['17'] = j.index[0]
answers['17']

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [55]:
j = prods.merge( movies, on='imdb_id', how='left' )
j = j.query( "prod == 'Paramount Pictures' " )
j = j[ j.profit == j.profit.min() ]
r = j
answers['18'] = r.original_title.iloc[0] + ' (' + r.imdb_id.iloc[0] + ')'
answers['18']

'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [56]:
d = movies.groupby(['year']).sum().sort_values(['year'],ascending=False)
answers['19'] = d.index[0]
answers['19']

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [57]:
j = prods.merge( movies, on='imdb_id', how='left' )
j = j[ j['prod'].str.contains('Warner Bros') ]
j = j.groupby('year')['profit'].sum().sort_values(ascending=False)
j = j.loc[[2014,2008,2012,2010,2015]].sort_values(ascending=False)
answers['20'] = j.index[0]
answers['20']

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [58]:
answers['21'] = movies.groupby(['month']).count().sort_values(['imdb_id'],ascending=False).index[0]
answers['21']

9

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [59]:
answers['22'] = movies.groupby(['month']).count().loc[ [8,7,6] ].imdb_id.sum()
answers['22']

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [60]:
j = dirs.merge( movies, on='imdb_id', how='left' )
j['winter'] = j['month'].apply( lambda x: x in [1,2,12])
j = j[ j['winter'] ].groupby('dir')['imdb_id'].count().sort_values(ascending=False)
answers['23'] = j.index[0]
answers['23']

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [61]:
j = prods.merge( movies, on='imdb_id', how='left' )
j = j.groupby(['prod'])['original_title_len'].mean().sort_values(ascending=False)
answers['24'] = j.index[0]
answers['24']

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [62]:
j = prods.merge( movies, on='imdb_id', how='left' )
j = j.groupby(['prod'])['overview_words_count'].mean().sort_values(ascending=False)
answers['25'] = j.index[0]
answers['25']

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [80]:
j = movies.sort_values(['vote_average'],ascending=False)
j = j.head( int(len(j)*0.01) )
# Возможные варианты ответов согласно условию задачи
cases = { '1': ['Inside Out','The Dark Knight', '12 Years a Slave']
        , '2': ['BloodRayne','The Adventures of Rocky & Bullwinkle'] 
        , '3': ['Batman Begins','The Lord of the Rings: The Return of the King','Upside Down']
        , '4': ['300','Lucky Number Slevin','Kill Bill: Vol. 1']
        , '5': ['Upside Down','Inside Out','Iron Man']
        }
matched_cases = []
for k,titles in cases.items():
    fails = 0
    for title in titles:
        inTop = len(j[ j.original_title.str.contains(title) ]) > 0
        if not inTop:
            fails = fails + 1
    if fails==0:
        matched_cases.append((k,titles))
for mc in matched_cases:
    print( 'matched ',mc)
    answers['26'] = mc

matched  ('1', ['Inside Out', 'The Dark Knight', '12 Years a Slave'])


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [76]:
# Возможные варианты ответов согласно условию
pairs = [
    { 'count': 0, 'names': ['Johnny Depp', 'Helena Bonham Carter'] },
    { 'count': 0, 'names': ['Ben Stiller', 'Owen Wilson'] },
    { 'count': 0, 'names': ['Vin Diesel', 'Paul Walker'] },
    { 'count': 0, 'names': ['Adam Sandler', 'Kevin James'] },
    { 'count': 0, 'names': ['Daniel Radcliffe', 'Rupert Grint'] },
]
for index, srow in raw_data.iterrows():
    actors = srow.cast.split('|')
    for p in pairs:
        matched = 0
        for name in p['names']:
            if name in actors:
                matched = matched + 1
        if matched==len(p['names']):
            p['count'] = p['count'] + 1

sorted_pairs = sorted(pairs, key=lambda p:p['count'])
answers['27'] = sorted_pairs[ len(sorted_pairs)-1 ]
answers['27']

{'count': 8, 'names': ['Daniel Radcliffe', 'Rupert Grint']}

# Submission

In [83]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 9,
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '27': {'count': 8, 'names': ['Daniel Radcliffe', 'Rupert Grint']},
 '26': ('1', ['Inside Out', 'The Dark Knight', '12 Years a Slave'])}

In [82]:
# и убедиться что ни чего не пропустил)
len(answers)

27